#### From SAV to love

# HW №1

Осуществим предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания. Для работы объединим train_df и test_df.

Задания:

1) Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию:

для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
для для замены @user на пробел, необходимо использовать re.sub() при применении функции необходимо использовать np.vectorize(function).
2) Изменим регистр твитов на нижний с помощью .lower().

3) Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

4) Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

5) Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

6) Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

7) Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

8) Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.

9) Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).

10) Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

11) Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

12) Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

13) Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

14) Сохраним результат предобработки в pickle-файл.

In [2]:
import pandas as pd
import re
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os
import pickle
import pymystem3
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

In [3]:
train_df = pd.read_csv('train_tweets.csv')
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
test_df = pd.read_csv('test_tweets.csv')
test_df.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [5]:
combine_df = train_df.append(test_df, ignore_index = True, sort = False)
combine_df.head()

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


In [6]:
m = MorphAnalyzer()

# убираем все небуквенные символы
regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

In [7]:
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]

In [10]:
tokens = words_only(combine_df.tweet[0])

In [11]:
mystopwords = stopwords.words('english')
def remove_stopwords(lemmas, stop_words = mystopwords):
    return [w for w in lemmas if w not in stop_words]

In [12]:
lemmas = lemmatize_text(tokens)

In [13]:
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 1]

In [14]:
def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return remove_stopwords(lemmas)

In [15]:
re_express = re.compile('@[\w]*')

In [16]:
combine_df['tweet']=combine_df['tweet'].str.replace(re_express, ' ')

In [17]:
combine_df['tweet']=combine_df['tweet'].str.lower()

In [18]:
re_express1 = re.compile('[^\w\s]')

In [19]:
combine_df['tweet']=combine_df['tweet'].str.replace(re_express1, ' ')

In [20]:
re_express2 = re.compile('[^a-zA-Z0-9]')

In [21]:
combine_df['tweet']=combine_df['tweet'].str.replace(re_express2, ' ')

In [22]:
re_express3 = re.compile('[^a-zA-Z]')

In [23]:
combine_df['tweet']=combine_df['tweet'].str.replace(re_express3, ' ')

In [24]:
combine_df['tweet']= combine_df['tweet'].str.findall('\w{1,}').str.join(' ')

In [26]:
from nltk.tokenize import TweetTokenizer
tt = TweetTokenizer()
combine_df['tweet_token'] = combine_df['tweet'].apply(tt.tokenize) 

In [27]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [28]:
combine_df['tweet_token_filtered'] = combine_df['tweet_token'].apply(lambda x: [item for item in x if item not in stop])

In [29]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [30]:
combine_df['tweet_stemmed'] = combine_df['tweet_token_filtered'].apply(lambda x: [stemmer.stem(y) for y in x])

In [31]:
combine_df.head(10)

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed
0,1,0.0,when a father is dysfunctional and is so selfi...,"[when, a, father, is, dysfunctional, and, is, ...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc..."
1,2,0.0,thanks for lyft credit i can t use cause they ...,"[thanks, for, lyft, credit, i, can, t, use, ca...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]"
3,4,0.0,model i love u take with u all the time in ur,"[model, i, love, u, take, with, u, all, the, t...","[model, love, u, take, u, time, ur]","[model, love, u, take, u, time, ur]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]"
5,6,0.0,huge fan fare and big talking before they leav...,"[huge, fan, fare, and, big, talking, before, t...","[huge, fan, fare, big, talking, leave, chaos, ...","[huge, fan, fare, big, talk, leav, chao, pay, ..."
6,7,0.0,camping tomorrow danny,"[camping, tomorrow, danny]","[camping, tomorrow, danny]","[camp, tomorrow, danni]"
7,8,0.0,the next school year is the year for exams can...,"[the, next, school, year, is, the, year, for, ...","[next, school, year, year, exams, think, schoo...","[next, school, year, year, exam, think, school..."
8,9,0.0,we won love the land allin cavs champions clev...,"[we, won, love, the, land, allin, cavs, champi...","[love, land, allin, cavs, champions, cleveland...","[love, land, allin, cav, champion, cleveland, ..."
9,10,0.0,welcome here i m it s so gr,"[welcome, here, i, m, it, s, so, gr]","[welcome, gr]","[welcom, gr]"


In [34]:
combine_df['tweet_lemmatized'] = lemmas

In [35]:
combine_df.to_pickle("./combine_df7.pkl")